In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.fraudulous import detect_honey_frauders
from src.constants import HONEYPOTS, LOG_FORMS_RESULTS_DIR
import pandas as pd
import Levenshtein
from pathlib import Path
import pickle as pk
from src.data.postprocessing import (get_emojis_voc_counts,write_emojis_voc_counts,
                                     display_whole_dir,generate_production_format)
import numpy as np
import pickle as pk

# Postprocessing

<div class="alert alert-success">
Postprocessing of data gathered from mturk/gform
</div>

In [3]:
# easy visualization of the gathered dataframes
display_whole_dir(Path("../data/processed/auto_mturk/forms_log_results/run4"))

,Timestamp,Worker ID,🔈,🥜,😆,📭,🎞️,☂️,💣,🛐,🌸,🛣️,📡,Feedback
0,02/12/2020 14:50:48,A30UE6DWFNUCWX,volume,peanut,laughing,mailbox,film,umbrella,bomb,praying,flower,highway,satellite,it was fun thanks so much
1,02/12/2020 14:51:58,AWN1PEZWRQSQ9,volume,peanut,laughter,mailbox,film,umbrella,bomb,church,flower,exit,satalight,NaN
2,02/12/2020 14:52:10,A18T7E73TNGOKP,speaker,peanut,laughing,mailbox,film,umbrella,bomb,pray,flower,freeway,satellite,no
3,02/12/2020 14:53:01,A2YO837C0O1E91,volume,peanut,laughing,mailbox,film,umbrella,bomb,pray,flower,highway,cable,NaN
4,02/12/2020 14:59:56,A2D2JX8R0QU9G4,speaker,peanut,laugh,mailbox,flim,umbrella,bomb,worship,flower,highway,antenna,Nothing
5,02/12/2020 15:01:44,A12OZ7P7Z41SJI,sound,peanuts,laugh,mail,video,rain,bomb,shelter,flower,highway,satelite,NaN
6,02/12/2020 15:02:19,A143XRCI1YXAFE,volume,peanut,laugh,mail,film,umbrella,bomb,pray,lilly,highway,satelite,NaN
7,02/12/2020 15:07:34,A2ALESYWNEZU1D,speaker,peanut,laugh,mailbox,flim,umbrella,bomb,worship,flower,highway,antenna,Nothing
8,02/12/2020 15:09:09,A13OMD8QDV0YON,sound,peanut,lfunny,mail,movie,umbrella,bomb,pray,flower,highway,signal,NaN
9,02/12/2020 16:16:47,AAQREZOK13OV7,sound,peanut,laughing,mail,film,umbrella,bomb,prayer,flower,driving,satellite,NaN


,Timestamp,Worker ID,👨‍💼,🛋️,🥴,🖲️,🤤,⭐,🥘,💊,🔨,😮,📟,Feedback
0,02/12/2020 14:50:37,AWN1PEZWRQSQ9,office,couch,crazy,space,drool,star,stewpot,pills,hammer,wow,pager,NaN
1,02/12/2020 14:52:51,A30UE6DWFNUCWX,employee,couch,nervous,knight,drooling,star,soup,pill,hammer,shocked,pager,thank you!
2,02/12/2020 14:53:25,A18T7E73TNGOKP,presentation,livingroom,drunk,ufo,drooling,star,food,pill,hammer,shocked,creditcard,NaN
3,02/12/2020 15:01:03,A345VDWYMDWTGD,stocks,furniture,drunk,button,drooling,star,skillet,pill,hammer,wow,card,NaN
4,02/12/2020 15:01:59,A2D2JX8R0QU9G4,manager,sofa,guilt,bot,vomit,star,cook,capsule,hammer,exclamation,calculator,Nothing
5,02/12/2020 15:10:40,A13OMD8QDV0YON,work,couch,confused,alien,funny,star,food,medicine,tool,surprised,pager,NaN
6,02/12/2020 15:11:32,A1PLZ8RM4NW43J,businessman,couch,awkward,button,drool,star,stew,pill,hammer,wow,pager,NaN
7,02/12/2020 15:12:11,A143XRCI1YXAFE,presentation,sofa,silly,button,yum,star,paella,pill,hammer,wow,pager,NaN
8,02/12/2020 15:23:38,A15OKHGB4RDA9P,happy,couch,sick,button,happy,star,lunch,tablet,hammer,wow,calculator,good
9,02/12/2020 15:44:40,A12OZ7P7Z41SJI,teacher,home,crazy,button,drool,star,dinner,medicine,hammer,ohh,debit,NaN


## Visualizing the vocabulary 

In [4]:
em2vocab = get_emojis_voc_counts(Path("../data/processed/auto_mturk/forms_log_results/"))

In [5]:
write_emojis_voc_counts(em2vocab,"./emvoc_500ass.txt")

## Generating final format

In [19]:
pd.read_csv("../data/processed/auto_mturk/forms_log_results/batchesrun6/0/0.csv")

,Timestamp,Worker ID,Age,Gender,Mothertongue,🔈,🥜,😆,📭,🎞️,☂️,💣,🛐,🌸,🛣️,📡,Feedback
0,11/12/2020 21:08:12,A27VFM67RPD2L5,49,Female,english,volume,peanut,joyous,mailbox,film,umbrella,bomb,church,flower,road,broadcast,NaN
1,11/12/2020 21:21:41,A18T7E73TNGOKP,33,Male,english,sound,peanut,funny,mail,film,umbrella,bomb,church,flower,drive,satellite,NaN
2,11/12/2020 21:31:20,A3HMVAA8IWSF0O,33,Male,english,speaker,peanut,smile,mailbox,filmroll,umbrella,bomb,meditation,flower,road,satellite,Good


In [17]:
df = generate_production_format(Path("../data/processed/auto_mturk/forms_log_results/batchesrun6/"))
df.to_csv("../data/processed/auto_mturk/prod_format/batchesrun6.csv")

100%|██████████| 9/9 [00:00<00:00, 365.16it/s]


In [18]:
df

,Worker ID,emoji_index,emoji,word
205,A2J51BI7XS5ZYU,8,↔️,crossing
215,A3S0IC3M8RZ54U,8,↔️,on
195,A3HMVAA8IWSF0O,8,↔️,arrow
88,A3HMVAA8IWSF0O,18,⌨️,keyboard
68,A27VFM67RPD2L5,18,⌨️,keyboard
...,...,...,...,...
103,A13OMD8QDV0YON,1281,🧤,clap
113,A3VLTC2Z0NPV7J,1281,🧤,hands
118,A3VLTC2Z0NPV7J,1313,🩸,blood
108,A13OMD8QDV0YON,1313,🩸,blood


In [13]:
df1 = generate_production_format(Path("../data/processed/auto_mturk/forms_log_results/demographicrun5//"))
pk.dump(df1,open("./demographic_pilot5.pk","wb"))

100%|██████████| 1/1 [00:00<00:00, 38.80it/s]
